In [23]:
import pandas as pd, numpy as np
import warnings, statsmodels.api as sm
from patsy import dmatrices
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('https://github.com/CausalML/TreatmentEffectRisk/raw/main/data/behaghel.csv')
data.head()

,Unnamed: 0,sw,A_public,A_private,A_standard,Y,College_education,nivetude2,Vocational,High_school_dropout,...,Sensitive_suburban_area,Insertion,Interim,Conseil,age,Number_of_children,exper,salaire.num,mois_saisie_occ,ndem
0,1,0.831709,0,1,0,0,0,0,0,1,...,0,1,0,0,48,0,10,2,12,3
1,2,0.827225,0,1,0,0,0,0,1,0,...,0,1,0,0,48,2,5,3,5,1
2,3,0.827225,0,1,0,0,0,0,1,0,...,0,1,0,0,42,2,25,4,6,3
3,4,0.957266,0,1,0,0,0,0,1,0,...,0,1,0,0,40,1,15,4,3,1
4,5,0.957266,0,1,0,0,0,0,1,0,...,0,1,0,0,38,0,6,4,3,7


In [9]:
job_binary = data.query(
    "A_public == 1 or A_private == 1"
)
job_binary['sw'] = job_binary['sw'] / job_binary['sw'].mean()
job_binary['A'] = job_binary['A_public']

# means

def mean_p(data: pd.DataFrame, col, ref = 'sw'):
    return  data[col] * (data[ref] * data[col]).mean()

job_binary['ipw'] = 1 / (
    mean_p(job_binary, 'A_standard') +
    mean_p(job_binary, 'A_private') +
    mean_p(job_binary, 'A_public')
)

In [17]:
def make_cvgroup(n, K, right = True):
    split = np.random.uniform(size = n)
    qw = np.quantile(split, np.linspace(0, 1, K + 1))

    cvgroup = pd.cut(
        split, bins = qw, include_lowest=True,
        right=right, labels=False
    )
    return cvgroup.astype(int)

def make_cvgroup_balanced(data, K, form_t):
    cvgroup = np.zeros(len(data), dtype=int)
    
    form_t_is_1 = data[form_t] == 1
    form_t_is_0 = data[form_t] == 0
    
    cvgroup[form_t_is_1] = make_cvgroup(np.sum(form_t_is_1), K, right=True)
    cvgroup[form_t_is_0] = make_cvgroup(np.sum(form_t_is_0), K, right=False)
    
    return cvgroup

In [25]:
Xbin = [
  'College_education',
  'nivetude2',
  'Vocational',
  'High_school_dropout',
  'Manager',
  'Technician',
  'Skilled_clerical_worker',
  'Unskilled_clerical_worker',
  'Skilled_blue_colar',
  'Unskilled_blue_colar',
  'Woman',
  'Married',
  'French',
  'African',
  'Other_Nationality',
  'Paris_region',
  'North',
  'Other_regions',
  'Employment_component_level_1',
  'Employment_component_level_2',
  'Employment_component_missing',
  'Economic_Layoff',
  'Personnal_Layoff',
  'End_of_Fixed_Term_Contract',
  'End_of_Temporary_Work',
  'Other_reasons_of_unemployment',
  'Statistical_risk_level_2',
  'Statistical_risk_level_3',
  'Other_Statistical_risk',
  'Search_for_a_full_time_position',
  'Sensitive_suburban_area',
  'Insertion',
  'Interim',
  'Conseil'
]

Xnum = [
  'age',  
  'Number_of_children', 
  'exper', # years experience on the job
  'salaire.num', # salary target
  'mois_saisie_occ', # when assigned
  'ndem' # Num. unemployment spell
]

Xall = Xbin + Xnum
# Xall

In [ ]:
def wtdquantile(y, w, g):
    if g >= 1:
        return np.max(y)
    
    o = np.argsort(y)
    
    y_sorted = y[o]
    w_sorted = w[o]
 
    cum_weights = np.cumsum(w_sorted)
    threshold_index = np.where(cum_weights >= np.sum(w) * g)[0][0]
    
    return y_sorted[threshold_index]

In [34]:
K = 5
cvgroup = make_cvgroup_balanced(job_binary, K, 'A')


mu0_pred = mu1_pred = len(job_binary)

job_binary['y_dep'] = (2 * job_binary['A'] - 1 ) * job_binary['ipw'] * job_binary['Y']

y = job_binary['y_dep']
X = job_binary[Xall]

model = sm.WLS(job_binary['y_dep'], job_binary[Xall], weights=job_binary['sw']).fit()
tau_pred = model.predict(X)
